# Start with importing

## Import packages and functions

In [15]:
import pandas as pd
import glob
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from textblob import TextBlob

import matplotlib.pyplot as plt

## Import data

In [67]:
# get data file names

filenames = glob.glob("Data\*.csv")

filenames_list = []
for filename in filenames:
    filenames_list.append(pd.read_csv(filename))
    
# Concatenate all data into one DataFrame
all_files_all_columns = pd.concat(filenames_list, ignore_index=True)

target_data = all_files_all_columns[['retweet_count', 'favorite_count','full_text','created_at']]

just_tweets_and_dates = all_files_all_columns[['full_text','created_at']]

## Gaussian Naive Bayes with TextBlob 

In [75]:
# clean tweets
!pip install tweet-preprocessor
import preprocessor as p
import preprocessor.api as p
from preprocessor.api import clean, tokenize, parse
import re

# this package remove Reserved words (RT, FAV),Emojis,Smileys
# https://towardsdatascience.com/basic-tweet-preprocessing-in-python-efd8360d529e

# I had to change some details to get preprocessor to install properly
# Something to bear in mind for when we push the code to the public - we need to test it thoroughly and make sure it loads well.

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [61]:
target_data.full_text=[p.clean(i) for i in target_data.full_text]

# remove puncutations 
target_data.full_text = [re.sub(r'[^\w\s]', '', i) for i in target_data.full_text]

# remove colon 
target_data.full_text=[re.sub(':', '',i) for i in target_data.full_text]

# extract month to build classification label 
target_data.created_at=[i[:7] for i in target_data.created_at]

target_data.head()

# cleaning the full text field and check the top few rows

,retweet_count,favorite_count,full_text,created_at
0,0,0,Crohns MAP Vaccine A cure for crohns,2017-04
1,16,0,This study finds no evidence of increased risk...,2017-04
2,5,0,Ive had a few questions about the vaccine toda...,2017-04
3,0,0,per completezza di informazione,2017-04
4,3,0,Positive public health effect Vaccine credited...,2017-04


In [124]:
# not relevant to JK's efforts - maintained from AZ's work

df1=df.copy()

df1['polarity']=[TextBlob(i).sentiment.polarity for i in df1.full_text] 
df1['subjectivity']=[TextBlob(i).sentiment.subjectivity for i in df1.full_text] 

df1=df1.drop(['full_text'], axis=1)

df1.head()

NameError: name 'df' is not defined

In [78]:
# not relevant to JK's efforts - maintained from AZ's work

X = df1.drop(columns=['created_at'], axis=1)
y = df1['created_at']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [79]:
# not relevant to JK's efforts - maintained from AZ's work

y_pred = model.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"    % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 2160 points : 1799


In [ ]:
# below is before text cleaning...surprisingly more accurate

In [30]:
# not relevant to JK's efforts - maintained from AZ's work

X = df2.drop(columns=['created_at'], axis=1)
y = df2['created_at']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [32]:
# not relevant to JK's efforts - maintained from AZ's work

y_pred = model.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"    % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 2160 points : 1786


###  Trying again directly with text

In [115]:
# Reduce the files to only two columns, clean them, inspect top rows

just_tweets_and_dates_df = all_files_all_columns[['full_text','created_at']]

just_tweets_and_dates_df.full_text=[p.clean(i) for i in just_tweets_and_dates_df.full_text]
just_tweets_and_dates_df.full_text = [re.sub(r'[^\w\s]', '', i) for i in just_tweets_and_dates_df.full_text]
just_tweets_and_dates_df.full_text=[re.sub(':', '',i) for i in just_tweets_and_dates_df.full_text]
just_tweets_and_dates_df.created_at=[i[:7] for i in just_tweets_and_dates_df.created_at]


just_tweets_and_dates_df.head()


c:\python39\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,full_text,created_at
0,Crohns MAP Vaccine A cure for crohns,2017-04
1,This study finds no evidence of increased ris...,2017-04
2,Ive had a few questions about the vaccine tod...,2017-04
3,per completezza di informazione,2017-04
4,Positive public health effect Vaccine credite...,2017-04


In [116]:
# separate into training and test sections, check length of each

jtad_train_df, jtad_test_df = train_test_split(just_tweets_and_dates_df, test_size=0.2, random_state=1)

print("training is ", len(jtad_train_df), " values long")
print("testing is ", len(jtad_test_df), " values long")

training is  8640  values long
testing is  2160  values long


In [121]:
#convert the data frames to lists as textblob.classifier does not take data frames

jtad_train_list = jtad_train_df.values.tolist()
jtad_test_list = jtad_test_df.values.tolist()

In [122]:
# Train a textblob classifier on the training set

from textblob.classifiers import NaiveBayesClassifier
cl = NaiveBayesClassifier(jtad_train_list)

In [123]:
# check accuracy on test set

cl.accuracy(jtad_test_list)

0.8138888888888889

In [ ]:
# now - try again with POS-tags? stem? stop words? lemmatisers? 

# Plus, try again with different training regims

# e.g. cl.update(new_training_data) and then repeat cl.accuraccy(test)

## Multinomial Naive Bayes with Tfidf

In [80]:
df2=df.copy()

In [81]:
tfidf_vectorizer = TfidfVectorizer()
df2['full_text'] = tfidf_vectorizer.fit_transform(df2['full_text']).toarray()

In [82]:
df2['full_text'].sum()

0.5592335247934099

In [93]:
X = df2.drop(columns=['created_at'], axis=1)
y = df2['created_at']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [95]:
y_pred = model.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"    % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 2160 points : 1940
